In [1]:
import matplotlib

import matplotlib.pyplot as plt

import numpy as np

import os

import torch

from scipy.io import loadmat

from tqdm import tqdm_notebook as tqdm

In [2]:
%matplotlib inline

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

In [4]:
# Add new methods here.
methods = ['sift_d2net_os8_512kpts', 'sift_d2net_os4_512kpts', 'sift_d2net_os2_512kpts', 'sift_d2net_os1_512kpts', 'sift_apd2net_os4_512kpts_dils_3', 'sift_apd2net_os4_512kpts_dils_5', 'sift_apd2net_os4_512kpts_dils_7', 'sift_apd2net_os4_512kpts_dils_11', 'sift_apd2net_os4_512kpts_dils_21', 'sift_apd2net_os4_512kpts_dils_31', 'sift_apd2net_os2_512kpts_dils_3_21', 'sift_apd2net_os2_512kpts_dils_5_21', 'sift_apd2net_os2_512kpts_dils_7_21', 'sift_apd2net_os2_512kpts_dils_11_21', 'sift_apd2net_os2_512kpts_dils_15_21', 'sift_apd2net_os2_512kpts_dils_21_21', 'sift_apd2net_os2_512kpts_dils_27_27', 'sift_apd2net_os2_512kpts_dils_31_31', 'sift_apd2net_os2_512kpts_dils_37_37', 'sift_apd2net_os2_512kpts_dils_11_37', 'sift_apd2net_os2_512kpts_dils_15_37', 'sift_apd2net_os2_512kpts_dils_17_37', 'sift_apd2net_os2_512kpts_dils_19_37', 'sift_apd2net_os2_512kpts_dils_21_37', 'sift_apd2net_os2_512kpts_dils_27_37', 'sift_apd2net_os2_512kpts_dils_19_31', 'sift_apd2net_os2_512kpts_dils_19_33', 'sift_apd2net_os2_512kpts_dils_19_35', 'sift_apd2net_os1_512kpts_dils_3_19_35', 'sift_apd2net_os1_512kpts_dils_5_19_35', 'sift_apd2net_os1_512kpts_dils_9_19_35', 'sift_apd2net_os1_512kpts_dils_11_19_35', 'sift_apd2net_os1_512kpts_dils_15_19_35', 'sift_apd2net_os1_512kpts_dils_19_19_35', 'sift_apd2net_os1_512kpts_dils_19_35_35', 'sift_apd2net_os1_512kpts_dils_19_41_41', 'sift_apd2net_os1_512kpts_dils_19_51_51', 'sift_apd2net_os1_512kpts_dils_25_51_51']




In [5]:
# Change here if you want to use top K or all features.
# top_k = 2000
top_k = None 

In [6]:
n_i = 52
n_v = 56

In [7]:
dataset_path = 'hpatches-sequences-release'

In [8]:
lim = [1, 15]
rng = np.arange(lim[0], lim[1] + 1)

In [9]:
def mnn_matcher(descriptors_a, descriptors_b):
    device = descriptors_a.device
    sim = descriptors_a @ descriptors_b.t()
    nn12 = torch.max(sim, dim=1)[1]
    nn21 = torch.max(sim, dim=0)[1]
    ids1 = torch.arange(0, sim.shape[0], device=device)
    mask = (ids1 == nn21[nn12])
    matches = torch.stack([ids1[mask], nn12[mask]])
    return matches.t().data.cpu().numpy()

In [10]:
def benchmark_features(read_feats):
    seq_names = sorted(os.listdir(dataset_path))

    n_feats = []
    n_matches = []
    seq_type = []
    i_err = {thr: 0 for thr in rng}
    v_err = {thr: 0 for thr in rng}

    for seq_idx, seq_name in tqdm(enumerate(seq_names), total=len(seq_names)):
        keypoints_a, descriptors_a = read_feats(seq_name, 1)
        n_feats.append(keypoints_a.shape[0])

        for im_idx in range(2, 7):
            keypoints_b, descriptors_b = read_feats(seq_name, im_idx)
            n_feats.append(keypoints_b.shape[0])

            matches = mnn_matcher(
                torch.from_numpy(descriptors_a).to(device=device), 
                torch.from_numpy(descriptors_b).to(device=device)
            )
            
            homography = np.loadtxt(os.path.join(dataset_path, seq_name, "H_1_" + str(im_idx)))
            
            pos_a = keypoints_a[matches[:, 0], : 2] 
            pos_a_h = np.concatenate([pos_a, np.ones([matches.shape[0], 1])], axis=1)
            pos_b_proj_h = np.transpose(np.dot(homography, np.transpose(pos_a_h)))
            pos_b_proj = pos_b_proj_h[:, : 2] / pos_b_proj_h[:, 2 :]

            pos_b = keypoints_b[matches[:, 1], : 2]

            dist = np.sqrt(np.sum((pos_b - pos_b_proj) ** 2, axis=1))

            n_matches.append(matches.shape[0])
            seq_type.append(seq_name[0])
            
            if dist.shape[0] == 0:
                dist = np.array([float("inf")])
            
            for thr in rng:
                if seq_name[0] == 'i':
                    i_err[thr] += np.mean(dist <= thr)
                else:
                    v_err[thr] += np.mean(dist <= thr)
    
    seq_type = np.array(seq_type)
    n_feats = np.array(n_feats)
    n_matches = np.array(n_matches)
    
    return i_err, v_err, [seq_type, n_feats, n_matches]

In [11]:
def summary(stats):
    seq_type, n_feats, n_matches = stats
    print('# Features: {:f} - [{:d}, {:d}]'.format(np.mean(n_feats), np.min(n_feats), np.max(n_feats)))
    print('# Matches: Overall {:f}, Illumination {:f}, Viewpoint {:f}'.format(
        np.sum(n_matches) / ((n_i + n_v) * 5), 
        np.sum(n_matches[seq_type == 'i']) / (n_i * 5), 
        np.sum(n_matches[seq_type == 'v']) / (n_v * 5))
    )

In [12]:
def generate_read_function(method, extension='ppm'):
    def read_function(seq_name, im_idx):
        aux = np.load(os.path.join(dataset_path, seq_name, '%d.%s.%s' % (im_idx, extension, method)))
        if top_k is None:
            return aux['keypoints'], aux['descriptors']
        else:
            assert('scores' in aux)
            ids = np.argsort(aux['scores'])[-top_k :]
            return aux['keypoints'][ids, :], aux['descriptors'][ids, :]
    return read_function

In [13]:
def sift_to_rootsift(descriptors):
    return np.sqrt(descriptors / np.expand_dims(np.sum(np.abs(descriptors), axis=1), axis=1) + 1e-16)
def parse_mat(mat):
    keypoints = mat['keypoints'][:, : 2]
    raw_descriptors = mat['descriptors']
    l2_norm_descriptors = raw_descriptors / np.expand_dims(np.sum(raw_descriptors ** 2, axis=1), axis=1)
    descriptors = sift_to_rootsift(l2_norm_descriptors)
    if top_k is None:
        return keypoints, descriptors
    else:
        assert('scores' in mat)
        ids = np.argsort(mat['scores'][0])[-top_k :]
        return keypoints[ids, :], descriptors[ids, :]

In [14]:
if top_k is None:
    cache_dir = 'cache'
else:
    cache_dir = 'cache-top'
if not os.path.isdir(cache_dir):
    os.mkdir(cache_dir)

In [15]:
errors = {}

In [16]:
for method in methods:
    output_file = os.path.join(cache_dir, method + '.npy')
    print(method)
    if method == 'hesaff':
        read_function = lambda seq_name, im_idx: parse_mat(loadmat(os.path.join(dataset_path, seq_name, '%d.ppm.hesaff' % im_idx), appendmat=False))
    else:
        if method == 'delf' or method == 'delf-new':
            read_function = generate_read_function(method, extension='png')
        else:
            read_function = generate_read_function(method)
    print(output_file)
    if os.path.exists(output_file):
        print('Loading precomputed errors...')
        errors[method] = np.load(output_file, allow_pickle=True)
    else:
        errors[method] = benchmark_features(read_function)
        np.save(output_file, errors[method])
    summary(errors[method][-1])

sift_d2net_os8_512kpts
cache/sift_d2net_os8_512kpts.npy
Loading precomputed errors...
# Features: 499.279321 - [147, 512]
# Matches: Overall 132.968519, Illumination 156.134615, Viewpoint 111.457143
sift_d2net_os4_512kpts
cache/sift_d2net_os4_512kpts.npy
Loading precomputed errors...
# Features: 499.279321 - [147, 512]
# Matches: Overall 136.772222, Illumination 158.942308, Viewpoint 116.185714
sift_d2net_os2_512kpts
cache/sift_d2net_os2_512kpts.npy
Loading precomputed errors...
# Features: 499.279321 - [147, 512]
# Matches: Overall 138.209259, Illumination 158.642308, Viewpoint 119.235714
sift_d2net_os1_512kpts
cache/sift_d2net_os1_512kpts.npy
Loading precomputed errors...
# Features: 499.279321 - [147, 512]
# Matches: Overall 137.135185, Illumination 158.196154, Viewpoint 117.578571
sift_apd2net_os4_512kpts_dils_3
cache/sift_apd2net_os4_512kpts_dils_3.npy
Loading precomputed errors...
# Features: 499.279321 - [147, 512]
# Matches: Overall 131.472222, Illumination 155.157692, Viewpoin

# Results

In [17]:
for method in methods:
    i_err, v_err, _ = errors[method]
    print(method)
    print((i_err[3] + v_err[3]) / ((n_i + n_v) * 5))

sift_d2net_os8_512kpts
0.5499676578993848
sift_d2net_os4_512kpts
0.6467065231410095
sift_d2net_os2_512kpts
0.6699172353219817
sift_d2net_os1_512kpts
0.6765393005101755
sift_apd2net_os4_512kpts_dils_3
0.5563377962786727
sift_apd2net_os4_512kpts_dils_5
0.574983823697742
sift_apd2net_os4_512kpts_dils_7
0.5856896197752974
sift_apd2net_os4_512kpts_dils_11
0.6084090837225523
sift_apd2net_os4_512kpts_dils_21
0.6372205807383049
sift_apd2net_os4_512kpts_dils_31
0.6463382116020037
sift_apd2net_os2_512kpts_dils_3_21
0.6364869574780929
sift_apd2net_os2_512kpts_dils_5_21
0.6359311171845183
sift_apd2net_os2_512kpts_dils_7_21
0.6357537805549773
sift_apd2net_os2_512kpts_dils_11_21
0.6384298143079048
sift_apd2net_os2_512kpts_dils_15_21
0.6425601664442725
sift_apd2net_os2_512kpts_dils_21_21
0.6468800436460583
sift_apd2net_os2_512kpts_dils_27_27
0.6575674400143419
sift_apd2net_os2_512kpts_dils_31_31
0.6632574299625779
sift_apd2net_os2_512kpts_dils_37_37
0.666815353544364
sift_apd2net_os2_512kpts_dils_11_